In [1]:
import os 
import tqdm
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../../data/'
group_dir_ = '../../data/groups/random'
folder = 'all (user)'
file_name = 'normalized_to_rating_filter_track_5_user_100.pkl'

In [2]:
predictions = pd.read_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:5]

,uid,tid,rating,count
26,0,9,4.75,15
37,0,15,2.50,3
45,0,22,3.50,6
64,0,28,2.75,4
75,0,33,0.25,1


In [3]:
predictions[:5]

,uid,tid,rating
0,0,9,0.847839
1,0,15,0.594322
2,0,22,0.446187
3,0,28,0.721104
4,0,33,0.597911


In [4]:
groups_n =[]
group_sizes = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)
    print(len(groups[0]))

2
3
4
5
6
7
8
9
10


In [5]:
num_user = len(test['uid'].unique())
num_user

953

In [6]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm(range(num_user)):
    prediction = predictions[predictions['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

In [7]:
evaluation = []
satisfication = 0
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/num_user)


0.2091290661070307


In [8]:
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    for i in range(len(r)):
        r[i] = 2**r[i] -1
    
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, r_max, k, method=0):
    dcg_max = dcg_at_k(r_max, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [9]:
precision_4 = []
precision_20 = []
precision_50 = []

recall_4 = []
recall_20 = []
recall_50 = []

nDCG_4 = []
nDCG_20 = []
nDCG_50 = []

In [10]:
# Precision and Recall
top_n_size = 50
top_n_lists_g = []
for groups in tqdm(groups_n):

    group_size = len(groups[0])
    top_n_lists = []

    for group in groups:
        rating_table = pd.DataFrame() 
        for member in group:
            prediction = predictions[predictions['uid'] == member].copy()
            if len(rating_table) == 0:
                rating_table = prediction
            else:
                rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
        rating_table['rating'] /= group_size 

        rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
        rating_table = rating_table[:top_n_size]
        top_n_list = []
        for _, row in rating_table.iterrows():
            top_n_list.append(row[0])
        top_n_lists.append(top_n_list)
    top_n_lists_g.append(top_n_lists)   

threshold = 0.0
for top_n_size in tqdm([50, 20, 4]):
    avg_precisions = []
    avg_recalls = []
    avg_nDCGs = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        recalls =[]
        nDCGs = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j][:top_n_size]
            
            test_g = pd.DataFrame()
            for uid in group:
                t = test[test['uid'] == uid]
                if len(test_g) == 0:
                    test_g = t
                else:
                    test_g = test_g.set_index('tid').add(t.set_index('tid'), fill_value=0).reset_index()
            test_g['rating'] /= len(group)
            
            high_rating = 0
            truth_rating = []
            for k in range(top_n_size):
                tid = top_n_list[k]
                t = test_g[test_g['tid'] == tid]
                if len(t) > 0:
                    high_rating += 1
                    truth_rating.append(t.iloc[0]['rating'])
                else:
                    truth_rating.append(0)

            precision = high_rating / top_n_size
            recall = high_rating / len(test_g)
            
            max_rating = test_g.sort_values(by=['rating'],  ascending=False)['rating'].values[:top_n_size]
            nDCG = ndcg_at_k(truth_rating, max_rating, top_n_size, method=1)
            
            precisions.append(precision)
            recalls.append(recall)
            nDCGs.append(nDCG)  
                
        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
        
        avg_recall = 0
        for recall in recalls:
            avg_recall += recall
        avg_recall /= len(recalls)
        avg_recalls.append(avg_recall)
        
        avg_nDCG = 0
        for nDCG in nDCGs:
            avg_nDCG += nDCG
        avg_nDCG /= len(nDCGs)
        avg_nDCGs.append(avg_nDCG)
        
    if top_n_size == 4:
        precision_4.append(avg_precisions)
    elif top_n_size == 20:
        precision_20.append(avg_precisions)
    elif top_n_size == 50:
        precision_50.append(avg_precisions)
        
    if top_n_size == 4:
        recall_4.append(avg_recalls)
    elif top_n_size == 20:
        recall_20.append(avg_recalls)
    elif top_n_size == 50:
        recall_50.append(avg_recalls)

    if top_n_size == 4:
        nDCG_4.append(avg_nDCGs)
    elif top_n_size == 20:
        nDCG_20.append(avg_nDCGs)
    elif top_n_size == 50:
        nDCG_50.append(avg_nDCGs)

In [11]:
for i in precision_4:
    print(i)

for i in precision_20:
    print(i)

for i in precision_50:
    print(i)

for i in recall_4:
    print(i)

for i in recall_20:
    print(i)

for i in recall_50:
    print(i)
    
for i in nDCG_4:
    print(i)

for i in nDCG_20:
    print(i)

for i in nDCG_50:
    print(i)

[0.40825688073394495, 0.5694444444444444, 0.5787037037037037, 0.627906976744186, 0.6388888888888888, 0.7580645161290323, 0.6574074074074074, 0.8541666666666666, 0.7976190476190477]
[0.3178899082568807, 0.4097222222222222, 0.4712962962962962, 0.5255813953488372, 0.5763888888888888, 0.6193548387096773, 0.6296296296296297, 0.7041666666666666, 0.6928571428571427]
[0.2620183486238531, 0.3425, 0.38888888888888895, 0.4600000000000002, 0.5022222222222221, 0.5419354838709679, 0.5666666666666667, 0.625, 0.6419047619047618]
[0.0014770173617098031, 0.0014416725478537192, 0.0010002944229732203, 0.0008798228081927657, 0.0008015309022325961, 0.0007410973719752077, 0.0005854559528754701, 0.0006931170286650478, 0.0005893560475828312]
[0.0056023034843538455, 0.004932154589243326, 0.004162100696957212, 0.0036364223662373025, 0.0034777328056925963, 0.003073327938768504, 0.002858266908564622, 0.002799754283317474, 0.0025415093724253783]
[0.011304972082441399, 0.01020938435075574, 0.008603269321239693, 0.00